# Parte 1 - RAG - Retrieval-Augmented Generation

### Objetivo: Desenvolver um sistema que responde perguntas sobre um conjunto de artigos científicos locais (PDFs), usando uma abordagem de Retrieval-Augmented Generation.






#ATIVIDADE

- Altere os pontos marcados com #TODO(tópico 5 e 6)
- Carregar seus próprios artigos e datasets em PDF (tópico 2).
- Usar modelo gratuito (nossa sugestão é o llama via groq).
- Avaliar respostas automaticamente com métricas de NLP.

**Observação 01:** cada aluno deve adaptar o código a um domínio específico da sua linha de pesquisa (ex: Engenharia de software, IHC, IA, robótica, etc) e comparar a performance.


**Observação 02:** Caso necessário, faça suas alterações no código, conforme os conceitos vistos em sala de aula, para adequar ao caso específico que esteja tratando.

In [1]:
!pip install PyPDF2

# 1. Upload dos PDFs
Você carregará os PDFs que gostaria que fossem analisados.

In [2]:
from google.colab import files
uploaded = files.upload()

import os
from PyPDF2 import PdfReader

# Cria pasta para os PDFs
os.makedirs("corpus", exist_ok=True)
for fname in uploaded.keys():
    os.rename(fname, os.path.join("corpus", fname))

print("PDFs carregados:", os.listdir("corpus"))

Saving Raissi2017-I.pdf to Raissi2017-I.pdf
PDFs carregados: ['Raissi2017-I.pdf', 'Weinan2017.pdf']


# 2. Leitura e extração do texto dos PDFs
A função abaixo irá gerar o corpus (que é uma lista de textos). Cada elemento do corpus é o texto de um PDF carregado anteriormente.

In [3]:
def load_papers(folder):
    corpus = []
    for file in os.listdir(folder):
        if file.endswith(".pdf"):
            reader = PdfReader(os.path.join(folder, file))
            text = ""
            for page in reader.pages:
                text = page.extract_text() or ""
                corpus.append(text)
    return corpus

texts = load_papers("corpus")
print(f"{len(texts)} chunks carregados.")

36 chunks carregados.


# 3. Embeddings - Criação
Usar o sentence-transformers para transformar os textos extraídos dos PDFs em embeddings. (Se colab pedir acesso, conceda)

In [4]:
from sentence_transformers import SentenceTransformer
import numpy as np

model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = model.encode(texts, convert_to_tensor=True)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


# 4. Função de Recuperação - (R)AG
### Implementar o mecanismo de busca vetorial. Aqui entra o retriever: busca semântica por similaridade de embeddings.

In [5]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def retrieve(query, texts, embeddings, top_k=2, max_chars=3000):
    """
    Recupera os textos mais relevantes limitando o tamanho total (max_chars)
    para não exceder o limite de tokens do modelo Groq.
    """
    query_emb = model.encode([query])
    scores = cosine_similarity(query_emb, embeddings)[0]
    top_indices = np.argsort(scores)[::-1][:top_k]

    results = []
    total_len = 0
    for i in top_indices:
        snippet = texts[i]
        if total_len + len(snippet) > max_chars:
            snippet = snippet[: max_chars - total_len]  # corta para caber no limite
        results.append(snippet)
        total_len += len(snippet)
        if total_len >= max_chars:
            break
    return results

# 5. Integrar com uma LLM - R(AG)

### O conteúdo recuperado é passado como contexto ao modelo llama-3.3:70b a partir do groq

In [6]:
!pip install --upgrade langchain langchain-core langchain-groq

In [7]:
from langchain_groq import ChatGroq

#TODO sigas os passos em https://groq.com/ para pegar sua chave: Developers -> Free API key
GROQ_API_KEY = input( "Cole sua chave Groq aqui e dê <enter>: ")

client = ChatGroq(
    model="llama-3.3-70b-versatile",  #TODO modelo gratuito
    api_key=GROQ_API_KEY,
    temperature=0.2
)

def generate_answer(query, context):
    prompt = f"""
Use o contexto abaixo para responder a pergunta com precisão científica.
Contexto: {context}
Pergunta: {query}
"""
    response = client.invoke(prompt).content
    return response

Cole sua chave Groq aqui e dê <enter>: 


# 6. Teste do modelo

In [8]:
query = "O que são PINNs?"
context = " ".join(retrieve(query, texts, embeddings))
answer = generate_answer(query, context)
print("\nResposta gerada:\n", answer)


Resposta gerada:
  4. The bottom panel of Figure 1 shows the
predicted solution along with the exact solution at dierent times. The
agreement between the predicted and exact solutions is excellent, and
the relative L2 error is measured at 6 :710 4. The relative L2 error
between the predicted and exact solutions is dened as
kfkL2 = 1
Z
T
0
Z
X
0
j f(x;t)j2 dxdt
1=2
:
(5)
The relative L2 error is a measure of the dierence between the pre-
dicted and exact solutions, and it is used to quantify the accuracy of
the predictions.
What is the relative L2 error between the predicted and exact solutions for the Burgers' equation when Nu = 100 and Nf = 10000? 

According to the table, the relative L2 error between the predicted and exact solutions for the Burgers' equation when Nu = 100 and Nf = 10000 is 6.7e-04.


# Parte 2 - Pesquisa na web - Web-based RAG ou Online RAG
Nesta seção, faremos uma prática de buscas de informações na web. O objetivo é dar ao modelo dados atualizados retirados de artigos na web. Neste exemplo, no Retrieval **(Recuperação)**, ao invés de buscarmos de um corpus de documentos ou banco de dados, buscaremos da web. O conteúdo extraído da página da web será usado para Aumentar **(Augment)** o prompt fornecido ao modelo. E por fim, o modelo usará o prompt para Gerar **(Generation)** uma resposta que será o resumo de um artigo.

Em resumo, iremos:
demonstrar um fluxo de RAG (Retrieval Augmented Generation) buscando informações na internet usando DuckDuckGo, extraindo o conteúdo de um artigo relevante e resumindo-o usando o LLM pelo Groq.

#ATIVIDADE

**Altere os pontos marcados com #TODO** no código para testar diferentes
resultados.

Por exemplo, se houver: QUANT_MAX_ARTIGOS = 5  # TODO
mude para 10, por exemplo e veja como muda a seleção de artigos.

**Escolha o artigo que será usado.**

Por padrão, search_results[0] pega apenas o primeiro. Você pode testar com outro índice para ver respostas diferentes.

**Volte à Parte 1 e repita a execução.**

Lá o código junta os resultados do retrieve() nos chunks e passa esse contexto para o LLM.

Assim, você consegue comparar como as alterações influenciam a resposta final do modelo.





## 1. Instalar bibliotecas necessárias

Instalar bibliotecas para buscar na web (DuckDuckGo) e para extrair o conteúdo de páginas web.


In [9]:
!pip install ddgs
!pip install beautifulsoup4
!pip install requests

##2. Realizar busca na web

Usar a ferramenta de busca para encontrar artigos relevantes com base em uma consulta do usuário. O duckduckgo (ddgs) faz o trabalho de buscar artigos na Internet, assim como o Google.


In [10]:
from ddgs import DDGS

ddgs = DDGS()
QUANT_MAX_ARTIGOS = 5

#TODO altere a query abaixo para o contexto que deseja. Use a sintaxe como a do exemplo colocado na query
query = "Machine Learning + Fluid Dynamics"
search_results = ddgs.text(query, max_results=QUANT_MAX_ARTIGOS)

print("Resultados da busca:")
for result in search_results:
    print(f"Título: {result['title']}")
    print(f"URL: {result['href']}")
    print(f"Descrição: {result['body']}\n")

Resultados da busca:
Título: Machine Learning in Fluid Dynamics (To be updated) | CFD WITH A
URL: https://caefn.com/machine-learning/fluid-dynamics
Descrição: ... machine learning techniques to (computational) fluid dynamics . ... 2 thoughts on “ Machine Learning in Fluid Dynamics (To be updated) ”

Título: Machine Learning Fluid Dynamics | Zenotech - Zenotech Ltd
URL: https://zenotech.com/news/machine-learning-for-fluid-dynamics-spotlight-on-professor-paola-cinnella/
Descrição: The field of fluid dynamics and machine learning is rapidly emerging as an interesting area of innovation. ... Machine Learning for Fluid Dynamics ...

Título: Bridging the gap between Machine Learning, AI and Fluid
URL: https://zenotech.com/news/bridging-the-gap-between-machine-learning-ai-and-fluid-dynamics/
Descrição: ... is one of the reasons that I am so optimistic about the newly created ERCOFTAC Special Interest Group (SIG) in Machine Learning for Fluid Dynamics ...

Título: A critical assessment of mach

##3. Extrair conteúdo do artigo

Acessar a URL do artigo retornado pela busca e extrair o texto principal.


In [11]:
import requests
import random
from bs4 import BeautifulSoup

article_text = None
if not search_results:
    print("Nenhuma URL encontrada para extração.")
else:
    #TODO altere search_results[0] para escolher aleatoriamente o artigo de 0 a QUANT_MAX_ARTIGOS-1
    #TODO é possível que os sites não permitam acessar o conteúdo, dando erro. Altere de search_results[0] para algum índice de site diferente de 0\
    #     para tentar baixar conteúdo de algum dos sites baixados.
    random_index = random.randint(0, len(search_results)-1)
    article_url = search_results[random_index]['href']
    print(f"Selected article (index {random_index}): {article_url}")

    try:
        response = requests.get(article_url, timeout=10) # Adicionado timeout para evitar travamentos
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')
            # Tentativa de encontrar o texto principal. Isso pode precisar de ajustes
            # dependendo da estrutura HTML dos sites.
            article_text = ""
            paragraphs = soup.find_all('p')
            for p in paragraphs:
                article_text += p.get_text() + "\n"

            if article_text:
                print(f"Conteúdo do artigo extraído da URL: {article_url}")
                print("Primeiros 500 caracteres do texto extraído:")
                print(article_text[:500])
            else:
                print(f"Não foi possível extrair texto principal da URL: {article_url}")

        else:
            print(f"Erro ao acessar a URL {article_url}. Código de status: {response.status_code}")
    except requests.exceptions.RequestException as e:
        print(f"Erro ao acessar a URL {article_url}: {e}")

print(len(article_text))

Selected article (index 3): https://arxiv.org/html/2508.13430v1
Conteúdo do artigo extraído da URL: https://arxiv.org/html/2508.13430v1
Primeiros 500 caracteres do texto extraído:
The fluid dynamics community has increasingly adopted machine learning to analyze, model, predict, and control a wide range of flows. These methods offer powerful computational capabilities for regression, compression, and optimization. In some cases, machine learning has even outperformed traditional approaches. However, many fluid mechanics problems remain beyond the reach of current machine learning techniques. As the field moves from its current state toward a more mature paradigm, this arti
33039


##4. Quebrando os chunks

Quebrar em chunks o texto extraído para utilizar com contexto. Utilizando outra abordagem.


In [12]:
!pip install langchain langchain-text-splitters

In [13]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Configura o Text Splitter
# Instancia o separador de texto com as suas especificações
text_splitter = RecursiveCharacterTextSplitter(
    # Tamanho máximo de cada chunk em caracteres
    chunk_size=1500,
    # Tamanho de sobreposição entre chunks.
    # Isso ajuda a manter o contexto entre os chunks adjacentes.
    chunk_overlap=250,
    # Separadores que o splitter tentará usar, em ordem:
    # 1. Parágrafos (\n\n)
    # 2. Novas linhas (\n)
    # 3. Espaços (' ')
    # 4. Caracteres vazios ('')
    separators=["\n\n", "\n", " ", ""],
    length_function=len # Função usada para medir o tamanho (len para caracteres)
)

# 3. Quebrar o texto
chunks = text_splitter.create_documents([article_text])

# 4. Imprimir os resultados para verificação

print(f"Número total de Chunks criados: **{len(chunks)}**\n")
print("-" * 50)

# Itera sobre os chunks (objetos Document do LangChain)
for i, chunk in enumerate(chunks):
    content = chunk.page_content
    print(f"*** CHUNK {i+1} (Tamanho: {len(content)} caracteres) ***")
    print(content)
    print("-" * 50)

Número total de Chunks criados: **31**

--------------------------------------------------
*** CHUNK 1 (Tamanho: 1482 caracteres) ***
The fluid dynamics community has increasingly adopted machine learning to analyze, model, predict, and control a wide range of flows. These methods offer powerful computational capabilities for regression, compression, and optimization. In some cases, machine learning has even outperformed traditional approaches. However, many fluid mechanics problems remain beyond the reach of current machine learning techniques. As the field moves from its current state toward a more mature paradigm, this article offers a critical assessment of the key challenges that must be addressed. Tackling these technical issues will not only deepen our understanding of flow physics but also expand the applicability of machine learning beyond fundamental research. We also highlight the importance of community-maintained datasets and open-source code repositories to accelerate pro